In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from snowflake.ml.ray.datasource import SFStageImageDataSource, SFStageTextDataSource


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


### Create a Data Source to read unstructured data

In [ ]:
# reading the image, resize the image to 256 x 256 to lower memory requirement and help performance
image_source = SFStageImageDataSource(
    stage_location = "@DATA_STAGE_RAY/images/",
    database = "ST_DB",
    schema = "ST_SCHEMA",
    image_size=(256, 256),
)

In [ ]:
# reading the label
# data is loaded after these two steps
# create pointer to data in stages
label_source = SFStageTextDataSource(
    stage_location = "@DATA_STAGE_RAY/labels/",
    database = "ST_DB",
    schema = "ST_SCHEMA",
)

In [ ]:
# configure logger and only log critical errors
import ray
import logging

def configure_ray_logger() -> None:
    #Configure Ray logging
    ray_logger = logging.getLogger("ray")
    ray_logger.setLevel(logging.CRITICAL)

    data_logger = logging.getLogger("ray.data")
    data_logger.setLevel(logging.CRITICAL)

    #Configure root logger
    logger = logging.getLogger()
    logger.setLevel(logging.CRITICAL)

    #Configure Ray's data context
    context = ray.data.DataContext.get_current()
    context.execution_options.verbose_progress = False
    context.enable_operator_progress_bars = False

configure_ray_logger()

### Load into a ray dataset

In [ ]:
# everything is lazy loaded
# turns data source into a dataset
image_ds = ray.data.read_datasource(image_source)

Now we can print the image dataset schema, which has the shape of the image we set earlier and the file name.

In [ ]:
print(image_ds.schema())

Lazy Loaded 1,500 images and print a couple of images which consist of an array of pixels and the file name

In [ ]:
print(f'Total load {image_ds.count()} images')
image_ds.show(2)

In [ ]:
# read the label dataset, use 6 workers to read from the stage concurrently
label_ds = ray.data.read_datasource(label_source, concurrency=6)

Print out the label dataset schema. The label dataset looks like:

```text
466 441 493 470 3
454 300 493 396 2
331 248 364 283 4
221 314 253 350 4
151 149 182 175 5
492 28 525 55 6
424 24 461 53 6
250 341 278 370 6
539 259 592 316 1
89 469 127 497 5
```

Each row is delimited by a newline character, so that doesn't need to be called out. It will be handled automatically.

In [ ]:
print(label_ds.schema())

In [ ]:
print(label_ds.show(1))

### Batch Process both datasets to include addition columns

**Image Dataset**: add a join key, encode the images, standardize image

**Label Dataset**: add a join key, interpret the labels

Function process_image adds grayscale image standardization, then encodes the image so it can be saved to a Snowflake table and then creates a join key from the filename pattern to be able to link an image to a label.

Finally, we take this function and run the ray dataset map method to apply the function to every row of the dataset.

This mapping again is happening lazily, which means you can have a massive dataset and allow ray to handle the processing in the most efficient way possible so you can focus on business logic.

In [ ]:
import numpy as np
from typing import Dict
import base64
import os

def process_image(row):
    # If grayscale (2D), convert to 3D
    img = row['image']
    if len(img.shape) == 2:
        row['image'] = np.stack([img] * 3, axis=-1)  # Duplicate grayscale channel 3 times

    encoded_image = base64.b64encode(row['image'])
    row['encoded_image'] = encoded_image

    fn = row['file_name']
    join_id = os.path.splitext(fn)[0].split('/')[-1]
    row['join_id'] = join_id
    return row

# processed_image_ds = image_ds.map_batches(convert_to_torch, concurrency=4)
processed_image_ds = image_ds.map(process_image)

`processed_image_ds` is a processed ray dataset. Image has been processed using function above (lazily) and we're now 

In [ ]:
# force trigger operation for 1 image
processed_image_ds.show(1)

Split the values from the label files to be able to store bounding box coordinates individually.

In [ ]:
import os

def expand_label_column(batch: pd.DataFrame) -> pd.DataFrame:
    xmin_list = []
    ymin_list = []
    xmax_list = []
    ymax_list = []
    class_list = []
    file_names = []
    ids = []
    
    # Process each row
    for _, row in batch.iterrows():
        # Split the text and convert to list
        values = row['text'].strip().split()
        
        # Ensure we have exactly 5 values
        if len(values) != 5:
            raise ValueError(f"Expected 5 values in text, but got {len(values)} values")
            
        # Add values to respective lists
        xmin_list.append(float(values[0]))
        ymin_list.append(float(values[1]))
        xmax_list.append(float(values[2]))
        ymax_list.append(float(values[3]))
        class_list.append(int(values[4]))
        file_name = row['file_name']
        file_names.append(file_name)
        ids.append(os.path.splitext(file_name)[0].split('/')[-1] + '_test')
    
    # Create new dataframe
    new_df = pd.DataFrame({
        'join_id': ids,
        'file_name': file_names,
        'xmin': xmin_list,
        'ymin': ymin_list,
        'xmax': xmax_list,
        'ymax': ymax_list,
        'class': class_list,
    })
    return new_df 

processed_label_ds = label_ds.map_batches(expand_label_column, concurrency=6, batch_format='pandas')

Print first record - now everything is nicely parsed.

In [ ]:
processed_label_ds.show(1)

### Merge image source and label source into a single dataset

We have two ways of achieving this: 1) if customer is more familiar with `pandas.Dataframe` and if the data fits into memory, then we can convert all data into pandas (or write into snowflake) and do the rest of the ops. 2) If the data does not fit into memory, we can directly leverage ray dataset to do the processing. 

**Note**: Ray dataset is not naturally architechted to support join ops, so it's better for to use other method (in memory / snowflake) to perform joins

#### Method 1: convert both ray datasets into pandas dataframe and perform joins

In [ ]:
# show how to convert a ray dataset to a panda dataframe
image_df = processed_image_ds.drop_columns(cols=['image']).to_pandas()

In [ ]:
# pandas - return first 5 rows
image_df.head()

Do the same thing for the label - convert ray dataset to a panda dataframe and print first 5 rows

In [ ]:
label_df = processed_label_ds.to_pandas()

In [ ]:
label_df.head()

In [ ]:
# perform merge 
merged_train_df = pd.merge(image_df, label_df, how='inner', on='join_id')


In [ ]:
merged_train_df.head()

## Save the Transformed Dataset to a snowflake table
Customer may also save the processed image dataset and label dataset into snowflake easily

In [ ]:
from snowflake.ml.ray.datasink import SnowflakeTableDatasink

session.use_role(role="SYSADMIN")
session.use_database(database="ST_DB")
session.use_schema(schema="ST_SCHEMA")

table_to_save = "RAY_DEMO_JAN21_IMAGE_DS"
datasink = SnowflakeTableDatasink(
    table_name=table_to_save,
    database = "ST_DB",
    schema = "ST_SCHEMA",
    auto_create_table=True,
    override=True,
)

In [ ]:
processed_image_ds.drop_columns(cols=['image']).write_datasink(datasink, concurrency=4)

In [ ]:
SELECT * FROM RAY_DEMO_JAN21_IMAGE_DS;

In [ ]:
table_to_save = "RAY_DEMO_JAN21_LABEL_DS"
datasink = SnowflakeTableDatasink(
    table_name=table_to_save,
    database = "ST_DB",
    schema = "ST_SCHEMA",
    auto_create_table=True,
    override=True,
)
processed_label_ds.write_datasink(datasink, concurrency=4)

In [ ]:
SELECT * FROM RAY_DEMO_JAN21_LABEL_DS;

In [ ]:
table_to_save = "RAY_DEMO_JAN21_COMINED_DS"
datasink = SnowflakeTableDatasink(
    table_name=table_to_save,
    database = "ST_DB",
    schema = "ST_SCHEMA",
    auto_create_table=True,
    override=True,
)
processed_label_ds.write_datasink(datasink, concurrency=4)

In [ ]:
SELECT * FROM RAY_DEMO_JAN21_COMINED_DS;